In [1]:
# Create base project directory
!mkdir -p /content/drive/MyDrive/ppe_detection
%cd /content/drive/MyDrive/ppe_detection

# Create directory structure
!mkdir -p config src

# Create empty files
!touch src/__init__.py
!touch src/config.py
!touch src/dataset.py
!touch src/train.py
!touch src/utils.py
!touch src/inference.py
!touch requirements.txt
!touch main.py

/content/drive/MyDrive/ppe_detection


In [30]:
%%writefile /content/drive/MyDrive/ppe_detection/config/data.yaml
train: /content/drive/MyDrive/ppe_detection/dataset/train
val: /content/drive/MyDrive/ppe_detection/dataset/valid
test: /content/drive/MyDrive/ppe_detection/dataset/test

nc: 12
names: ['glove', 'goggles', 'helmet', 'mask', 'no-suit', 'no_glove', 'no_goggles', 'no_helmet', 'no_mask', 'no_shoes', 'shoes', 'suit']

roboflow:
  workspace: personal-protective-equipment
  project: ppes-kaxsi
  version: 8
  license: CC BY 4.0
  url: https://universe.roboflow.com/personal-protective-equipment/ppes-kaxsi/dataset/8

Overwriting /content/drive/MyDrive/ppe_detection/config/data.yaml


In [5]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="fGbCADJ8D90SfaKuRfxB")
project = rf.workspace("personal-protective-equipment").project("ppes-kaxsi")
dataset = project.version(8).download("yolov5", location="/content/drive/MyDrive/ppe_detection/dataset")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /content/drive/MyDrive/ppe_detection/dataset in yolov5pytorch:: 100%|██████████| 49860/49860 [00:09<00:00, 5099.90it/s]


In [6]:
%%writefile requirements.txt
ultralytics
torch
torchvision
opencv-python
pandas
numpy
albumentations
wandb
PyYAML

Overwriting requirements.txt


In [7]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 23.2 MB/s eta 0:00:00


In [41]:
%%writefile /content/drive/MyDrive/ppe_detection/src/config.py
from dataclasses import dataclass
from typing import List
import yaml

@dataclass
class ModelConfig:
    img_size: int = 416
    batch_size: int = 16
    epochs: int = 20
    model_type: str = 'yolov8m.pt'
    lr0: float = 0.01
    weight_decay: float = 0.0005
    patience: int = 20
    classes: List[str] = ('glove', 'goggles', 'helmet', 'mask', 'no-suit',
                         'no_glove', 'no_goggles', 'no_helmet', 'no_mask',
                         'no_shoes', 'shoes', 'suit')

# Notice this function is NOT indented under the class
def load_config(yaml_path='/content/drive/MyDrive/ppe_detection/config/data.yaml'):
    with open(yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    return ModelConfig(), data_config

Overwriting /content/drive/MyDrive/ppe_detection/src/config.py


In [31]:
%%writefile /content/drive/MyDrive/ppe_detection/src/dataset.py
from pathlib import Path
import cv2
from torch.utils.data import Dataset
from collections import Counter

class PPEDataset(Dataset):
    def __init__(self, img_dir, label_dir, transform=None):
        self.img_dir = Path(img_dir)
        self.label_dir = Path(label_dir)
        self.transform = transform
        self.image_files = list(self.img_dir.glob('*.jpg'))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        label_path = self.label_dir / f"{img_path.stem}.txt"

        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        labels = []
        if label_path.exists():
            with open(label_path, 'r') as f:
                labels = [line.strip().split() for line in f]

        if self.transform:
            transformed = self.transform(image=image, bboxes=labels)
            image = transformed['image']
            labels = transformed['bboxes']

        return image, labels

def analyze_dataset(data_path: str, classes):
    data_path = Path(data_path)
    labels_path = data_path / 'labels'
    class_distribution = Counter()

    for label_file in labels_path.glob('*.txt'):
        with open(label_file, 'r') as f:
            for line in f:
                class_id = int(line.split()[0])
                class_distribution[class_id] += 1

    print("\nClass Distribution")
    for class_id, count in class_distribution.items():
        print(f"Class {classes[class_id]}: {count}")

    return class_distribution

Overwriting /content/drive/MyDrive/ppe_detection/src/dataset.py


In [29]:
%%writefile /content/drive/MyDrive/ppe_detection/src/utils.py
from pathlib import Path

def verify_dataset(data_path: str):
    data_path = Path(data_path)

    # Check if the main directory exists
    if not data_path.exists():
        raise ValueError(f"Dataset directory does not exist: {data_path}")

    # Check for images and labels directories
    images_path = data_path / 'images'
    labels_path = data_path / 'labels'

    if not images_path.exists():
        raise ValueError(f"Missing directory: {images_path}")
    if not labels_path.exists():
        raise ValueError(f"Missing directory: {labels_path}")

    # Verify matching images and labels
    image_files = set(f.stem for f in images_path.glob('*.jpg'))
    label_files = set(f.stem for f in labels_path.glob('*.txt'))

    if image_files != label_files:
        print(f"Warning: Mismatched images and labels in {data_path}")

    return True

Overwriting /content/drive/MyDrive/ppe_detection/src/utils.py


In [47]:
%%writefile /content/drive/MyDrive/ppe_detection/src/train.py
from ultralytics import YOLO
import torch
import wandb
import yaml
from pathlib import Path

def train_model(config, data_config):
    wandb.init(project='PPE-Detection', config=vars(config))

    # Create a temporary YAML file with the data configuration
    yaml_path = Path('temp_data_config.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(data_config, f)

    model = YOLO(config.model_type)

    results = model.train(
        data=str(yaml_path),  # Pass the path to the YAML file as string
        epochs=config.epochs,
        imgsz=config.img_size,
        batch=config.batch_size,
        patience=config.patience,
        lr0=config.lr0,
        weight_decay=config.weight_decay,
        cache=True,
        device='0' if torch.cuda.is_available() else 'cpu',
        project='PPE-Detection',
        name='PPE_Detector',
        exist_ok=True,
        pretrained=True,
        optimizer='Adam',
        resume = True
    )

    # Clean up temporary file
    yaml_path.unlink(missing_ok=True)

    return model, results

def validate_model(model, valid_path):
    results = model.val(data=valid_path)
    return results

Overwriting /content/drive/MyDrive/ppe_detection/src/train.py


In [12]:
%%writefile /content/drive/MyDrive/ppe_detection/src/inference.py

import cv2
from ultralytics import YOLO

def inference_pipeline(model_path: str, image_path: str, conf_thresh: float = 0.25):
  model = YOLO(model_path)

  results = model.predict(
      source = image_path,
      conf = conf_thresh,
      iou = 0.45,
      show = True,
      save = True
  )
  return results

def process_results(results, image, classes):
  processed_image = image.copy()

  for result in results:
    boxes = result.boxes
    for box in boxes:
      x1, y1, x2, y2 = box.xyxy[0]
      conf = box.conf[0]
      cls_id = int(box.cls[0])
      cls_name = classes[cls_id]


      cv2.rectangle(processed_image,
                    (int(x1), int(y1)),
                    (int(x2), int(y2)),
                    (0, 255, 0), 2)

      cv2.putText(processed_image,
                  f"{cls_name} {conf:.2f}",
                  (int(x1), int(y1 - 10)),
                  cv2.FONT_HERSHEY_SIMPLEX,
                  0.9, (0, 255, 0), 2)



    return processed_image

Overwriting /content/drive/MyDrive/ppe_detection/src/inference.py


In [45]:
%%writefile /content/drive/MyDrive/ppe_detection/main.py
import sys
sys.path.append('/content/drive/MyDrive/ppe_detection')

from src.config import load_config
from src.dataset import analyze_dataset
from src.utils import verify_dataset
from src.train import train_model, validate_model
from src.inference import inference_pipeline, process_results
import wandb
import cv2

def main():
    # Load configurations
    config, data_config = load_config()

    # Verify and analyze dataset
    verify_dataset(data_config['train'])
    class_distribution = analyze_dataset(data_config['train'], config.classes)

    # Initialize wandb
    wandb.init(project="ppe-detection")

    try:
        # Train model
        model, training_results = train_model(config, data_config)

        # Save model
        model.save('/content/drive/MyDrive/ppe_detection/best_ppe_model.pt')

        # Example inference
        test_image_path = data_config['test'] + '/images/005303_jpg.rf.d88d9335996cf880d42a0754273536db.jpg'
        original_image = cv2.imread(test_image_path)
        if original_image is None:
            raise ValueError(f"Could not load image from {test_image_path}")

        test_results = inference_pipeline(
            model_path='best_ppe_model.pt',
            image_path=test_image_path
        )

        # Process and visualize results
        processed_image = process_results(test_results[0], original_image, model.names)

        # Save processed image
        cv2.imwrite('/content/drive/MyDrive/ppe_detection/test_results.jpg', processed_image)

        from google.colab.patches import cv2_imshow
        cv2_imshow(processed_image)

    finally:
        wandb.finish()

if __name__ == "__main__":
    main()

Overwriting /content/drive/MyDrive/ppe_detection/main.py


In [48]:
!python /content/drive/MyDrive/ppe_detection/main.py


Class Distribution
Class goggles: 8076
Class no_glove: 13635
Class glove: 10981
Class no_goggles: 7891
Class no_shoes: 60
Class shoes: 42
Class no-suit: 744
Class suit: 363
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: nikhilgeddam75 (nikhilgeddam75-nikhil). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.5
wandb: Run data is saved locally in /content/drive/MyDrive/ppe_detection/wandb/run-20241023_113031-fnzblsnr
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run lunar-lake-6
wandb: ⭐️ View project at https://wandb.ai/nikhilgeddam75-nikhil/ppe-detection
wandb: 🚀 View run at https://wandb.ai/nikhilgeddam75-nikhil/ppe-detection/runs/fnzblsnr
Ultralytics 8.3.20 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=temp_data_config.yaml, epochs=500, time=None, patienc

In [50]:
from ultralytics import YOLO
model = YOLO('/content/drive/MyDrive/ppe_detection/PPE-Detection/PPE_Detector/weights/best.pt')

In [53]:
results = model.predict(source='/content/drive/MyDrive/ppe_detection/dataset/test/images/005384_jpg.rf.555416d9ce829859bf928b3b599d7d0c.jpg', conf=0.25, save=True)


image 1/1 /content/drive/MyDrive/ppe_detection/dataset/test/images/005384_jpg.rf.555416d9ce829859bf928b3b599d7d0c.jpg: 416x416 (no detections), 22.5ms
Speed: 1.3ms preprocess, 22.5ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 416)
Results saved to runs/detect/predict


In [54]:
results = model.val(data='/content/drive/MyDrive/ppe_detection/config/data.yaml')

Ultralytics 8.3.20 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


val: Scanning /content/drive/MyDrive/ppe_detection/dataset/valid/labels.cache... 3570 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3570/3570 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 224/224 [00:43<00:00,  5.20it/s]


                   all       3570       7710      0.264      0.359      0.247      0.132
                 glove        448        984      0.799      0.873      0.881      0.461
               goggles       1121       1192       0.84      0.563      0.656      0.395
                helmet        237        283          0          0          0          0
                  mask        253        253          0          0          0          0
               no-suit         13         15     0.0138      0.733     0.0579     0.0506
              no_glove        789       1548       0.67      0.652      0.626       0.29
            no_goggles       1161       1384      0.794      0.482      0.631      0.314
             no_helmet        205        229          0          0          0          0
               no_mask        640        640          0          0          0          0
              no_shoes        291        525    0.00618    0.00571    0.00576    0.00165
                 shoe